In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display, clear_output

/home/john/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load data
df = pd.read_excel('test.xlsx')
#list(df)

In [5]:
df.head(10)

,ID,Name,Manufacturer,Silica type,H,S*,A,B,C (pH 2.8),C (pH 7.0),EB retention factor,USP type,Phase type,PC1,PC2
0,93,Acclaim 120 C18,Dionex,B,1.030,0.010,-0.140,-0.020,0.080,0.000,10.10,L1,C18,-0.976386,0.646713
1,94,Acclaim 120 C8,Dionex,B,0.850,0.000,-0.270,0.010,0.080,0.010,6.00,L7,C8,0.074983,0.391974
2,628,Acclaim C30,Thermo/Hypersil,B,0.973,-0.022,-0.126,0.004,0.353,0.320,5.52,L62,C30,-0.743032,-0.059834
3,756,Acclaim Mixed-Mode HILIC-1,Thermo/Hypersil,B,0.517,-0.086,0.000,0.126,0.067,0.893,1.70,L60,EP,1.571240,-0.837081
4,757,Acclaim Mixed-Mode WCX-1,Thermo/Hypersil,B,0.432,-0.099,-0.606,0.168,-0.156,0.990,1.50,L60,EP,3.538626,-0.337766
5,97,Acclaim Organic Acid,Dionex,other,0.830,-0.060,-0.380,0.000,-0.310,0.340,6.70,,Other,0.857792,0.297208
6,629,Acclaim Phenyl-1,Thermo/Hypersil,phenyl,0.689,-0.150,-0.548,0.068,0.013,0.150,4.27,L11,Phenyl,2.193622,-0.953418
7,96,Acclaim Polar Advantage,Dionex,EP,0.850,-0.060,-0.110,0.020,-0.270,0.350,6.80,L60,EP,0.301811,0.187884
8,98,Acclaim PolarAdvantage II,Dionex,EP,0.740,0.010,-0.550,0.210,-0.220,0.670,6.30,L60,EP,2.189617,1.497586
9,95,Acclaim300 C18,Dionex,B,0.950,-0.010,-0.170,0.010,0.260,0.220,2.90,L1,C18,-0.536415,0.183463


In [3]:
# We will only use the following columns
numeric_cols = ["H", "S*", "A", "B", "C (pH 2.8)"]

# Standardize the data
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df[numeric_cols])

# Apply PCA
pca = PCA(n_components=2)  # Here we choose to keep 2 components
principal_components = pca.fit_transform(scaled_df)

# Add the principal components to the dataframe
df['PC1'] = principal_components[:, 0]
df['PC2'] = principal_components[:, 1]

In [4]:
# Define the dropdowns with 'All' option
manufacturer_dropdown = widgets.Dropdown(options = ['All'] + list(df['Manufacturer'].unique()), description='Manufacturer:')
silica_dropdown = widgets.Dropdown(options = ['All'] + list(df['Silica type'].unique()), description='Silica type:')
usp_dropdown = widgets.Dropdown(options = ['All'] + list(df['USP type'].unique()), description='USP type:')
phase_dropdown = widgets.Dropdown(options = ['All'] + list(df['Phase type'].unique()), description='Phase type:')

# Define the text input for search
search_input = widgets.Text(description='Search:')

# Define a button for the search
search_button = widgets.Button(description='Press to search')

# Update the update_plot function to use the search input
def update_plot(*args):
    clear_output(wait=True)  # Clear the previous plot

    # Display the dropdowns and the search input
    display(manufacturer_dropdown, silica_dropdown, usp_dropdown, phase_dropdown, search_input, search_button)
    
    filtered_df = df.copy()

    # Apply filters if 'All' is not selected
    if manufacturer_dropdown.value != 'All':
        filtered_df = filtered_df[filtered_df['Manufacturer'] == manufacturer_dropdown.value]
    if silica_dropdown.value != 'All':
        filtered_df = filtered_df[filtered_df['Silica type'] == silica_dropdown.value]
    if usp_dropdown.value != 'All':
        filtered_df = filtered_df[filtered_df['USP type'] == usp_dropdown.value]
    if phase_dropdown.value != 'All':
        filtered_df = filtered_df[filtered_df['Phase type'] == phase_dropdown.value]
    
    # Highlight the data based on the search input
    if search_input.value:
        filtered_df['Search result'] = filtered_df['Name'].str.contains(search_input.value, case=False)
    else:
        filtered_df['Search result'] = False
    
    fig = px.scatter(filtered_df, x='PC1', y='PC2', hover_data=['Name'], color='Search result')
    fig.show()

# Add an observer to the search button click event
search_button.on_click(update_plot)

# Add observers to the dropdowns
manufacturer_dropdown.observe(update_plot, 'value')
silica_dropdown.observe(update_plot, 'value')
usp_dropdown.observe(update_plot, 'value')
phase_dropdown.observe(update_plot, 'value')

# Display the initial plot
update_plot()


Dropdown(description='Manufacturer:', options=('All', 'Dionex', 'Thermo/Hypersil', 'ACT', 'Waters', 'SepaChrom…

Dropdown(description='Silica type:', options=('All', 'B', 'other', 'phenyl', 'EP', 'F', 'CN', 'A', 'Other', 'C…

Dropdown(description='USP type:', options=('All', 'L1', 'L7', 'L62', 'L60', '\xa0', 'L11', 'L26', 'L43', 'L10'…

Dropdown(description='Phase type:', options=('All', 'C18', 'C8', 'C30', 'EP', 'Other', 'Phenyl', 'C4', 'phenyl…

Text(value='-', description='Search:')

Button(description='Press to search', style=ButtonStyle())